## 환경설정

In [23]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

from utils import *

# print(html)

### KBS NEWS

In [27]:
url = "https://news.kbs.co.kr/news/pc/main/main.html"
html = get_html(url)

news_titles = html.select_one('div.head-line p.title')
len(news_titles)

titles = [title.text.strip() for title in news_titles]
for title in titles:
    print(title)


[속보] 코스피시장 매수 사이드카 발동…

작년 8월 이후 8개월만


### QUOTES

In [ ]:
class quote:
    def __init__(self, text, author, tags):
        self.__text: str = text
        self.__author: str = author
        self.tags: list[str] = tags
    
    @property
    def text(self):
        return self.__text
    
    @text.setter
    def text(self, text):
        self.__text = text
    
    @property
    def author(self):
        return self.__author
    
    @author.setter
    def author(self, author):
        self.__author = author

    def __str__(self):
        return f"{self.text} by {self.author}"
    
    def __repr__(self):
        return f"{self.text} by {self.author}"

url = "https://quotes.toscrape.com/"
html = get_html(url)
quote_list: list[quote] = []
quote_list2: list[dict] = []

elements = html.select('div.quote')
for element in elements:
    quote_text = element.select_one('.text').text
    quote_author = element.select_one('.author').text
    quote_tags = element.select('.tag')
    tags = [tag.text for tag in quote_tags]

    quote_list.append(quote(text=quote_text, author=quote_author, tags=tags))
    quote_list2.append({"text": quote_text, "author": quote_author, "tags": tags})

['change', 'deep-thoughts', 'thinking', 'world']
['abilities', 'choices']
['inspirational', 'life', 'live', 'miracle', 'miracles']
['aliteracy', 'books', 'classic', 'humor']
['be-yourself', 'inspirational']
['adulthood', 'success', 'value']
['life', 'love']
['edison', 'failure', 'inspirational', 'paraphrased']
['misattributed-eleanor-roosevelt']
['humor', 'obvious', 'simile']


### 실습1

In [69]:
import time

from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

service = Service(ChromeDriverManager().install())

In [ ]:
driver = webdriver.Chrome(service=service)

# driver는 컨트롤러
driver.get("https://naver.com")
driver.maximize_window()
time.sleep(1)
driver.get("https://google.com")
time.sleep(1)
driver.back()
time.sleep(1)
driver.refresh()
time.sleep(1)
driver.quit()

### 실습2

In [71]:
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

driver = webdriver.Chrome(service=service)
driver.get("https://naver.com")

try:
    elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '#shortcutArea span.service_name'))
    )

    for element in elements:
        print(element.text)
    
except Exception as e:
    print(e)
    driver.quit()

time.sleep(60)
driver.quit()

메일
카페
블로그
스토어
뉴스
증권
부동산
지도
웹툰
치지직


In [72]:
# from selenium.webdriver.common.by import By
# driver = webdriver.Chrome(service=service)

# driver.get("https://naver.com")
# time.sleep(1)

# shortcuts = driver.find_element(By.CSS_SELECTOR, '#shortcutArea')
# services = shortcuts.find_elements(By.CSS_SELECTOR, '.service_name')

# for service in services:
#     print(service.text)

# time.sleep(60)
# driver.quit()

### 실습3

In [ ]:
import asyncio
from utils import *

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.remote.webdriver import WebElement


async def find_elements(driver: webdriver.Chrome, pattern: str) -> WebElement:
    elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, pattern))
    )

    return elements

driver = webdriver.Chrome(service=service)
driver.get("https://naver.com")

try:
    task1 = asyncio.create_task(async_quit(driver, 10))
    task2 = asyncio.create_task(find_elements(driver, '#shortcutArea span.service_name'))
    
    await task1
    await task2

    services = task2.result()

    for service in services:
        if service.text == "뉴스":
            service.click()
    
    # driver.quit()

except Exception as e:
    print(e)
    driver.quit()

name 'async_quit' is not defined


### 실습4

In [ ]:
import asyncio
# from utils import async_quit

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.remote.webdriver import WebElement

async def search_query(driver: webdriver.Chrome, keyword: str) -> WebElement:
    print("Async B")
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, '#query.search_input'))
    )
    element.send_keys(keyword)
    element.send_keys(Keys.ENTER)
    print("B finished")

async def async_quit(driver: webdriver.Chrome, t):
    print("Async A")
    await asyncio.sleep(t)
    driver.quit()
    print("A finished")

driver = webdriver.Chrome(service=service)
driver.get("https://naver.com")

try:
    task1 = asyncio.create_task(async_quit(driver, 10))
    task2 = asyncio.create_task(search_query(driver, keyword="날씨"))
    
    await task1
    await task2

except Exception as e:
    print(e)
    driver.quit()

Async A
Async B
B finished
A finished


### 실습7

In [ ]:
import asyncio

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys

async def get_review(driver: webdriver.Chrome, name: str) -> str:
    search_box = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.input_box .input_search'))
    )
    search_box.send_keys(name)
    search_box.send_keys(Keys.ENTER)
    
    element = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, '.place_section.no_margin'))
    )

    print(element)
    review = element[0].text
    print(review)
    
    return review

async def async_quit(driver: webdriver.Chrome, t):
    await asyncio.sleep(t)
    driver.quit()

driver = webdriver.Chrome(service=service)
driver.get("https://map.naver.com/")

try:
    task1 = asyncio.create_task(async_quit(driver, 20))
    task2 = asyncio.create_task(get_review(driver, name="보들이쭈꾸미"))
    
    await task1
    await task2

except Exception as e:
    print(e)
    driver.quit()

Message: 

local variable 'element' referenced before assignment
